In [1]:
!pip install --quiet sense2vec==1.0.3
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
!tar -xvf  s2v_reddit_2015_md.tar.gz
!ls s2v_old
!pip install tika
!pip install aspose-words==22.10.0

     |████████████████████████████████| 4.8 MB 8.6 MB/s 
     |████████████████████████████████| 1.0 MB 37.5 MB/s 
     |████████████████████████████████| 208 kB 46.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.8 which is incompatible.
confection 0.0.3 requires srsly<3.0.0,>=2.4.0, but you have srsly 1.0.6 which is incompatible.
--2022-11-17 11:54:45--  https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/50261113/52126080-0993-11ea-8190-8f0e295df22a?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-C

In [2]:
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk('s2v_old')
import en_core_web_sm
import random
import spacy.cli
spacy.cli.download("en_core_web_sm")
spacy_nlp = en_core_web_sm.load()
import numpy as np
from textblob import TextBlob

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
from nltk.corpus import wordnet
import nltk
from nltk import tokenize
from nltk.tokenize import sent_tokenize
import tika
from tika import parser
import aspose.words as aw
import itertools
from datetime import datetime
from multiprocessing import Pool
from multiprocessing import cpu_count
print(cpu_count)
from concurrent.futures import ProcessPoolExecutor
from time import sleep
import time
import nltk
nltk.download('punkt')

/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (3.4.1) requires spaCy v3.4 and is incompatible with the current spaCy version (2.3.8). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


<bound method BaseContext.cpu_count of <multiprocessing.context.DefaultContext object at 0x7fda08e35bd0>>


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
start = time.time()

In [4]:
def Preprocessing(task1):
  
  sentences = sent_tokenize(task1)
  sentence_to_words=[]
  for i in sentences[0:]:
   sentence_to_words.extend(i.split())  
  tokens = [ w for w in sentence_to_words if w[0]!='[' and w[-1]!= ']' ]
  import string
  remove = string.punctuation
  import re 
  remove = remove.replace(".", "").replace("-", "").replace(",", "").replace("%", "").replace("/", "")
  pattern = r"[{}]".format(re.escape(remove))
  table = str.maketrans('', '', pattern)
  stripped = [w.translate(table) for w in tokens]
  words_to_sentense=' '.join(stripped)
  words_to_sentense
  sentences_new = sent_tokenize(words_to_sentense)
  filtered_list_short_sentences = [sent for sent in sentences_new if len(sent)>30 and len(sent)<250]
  return filtered_list_short_sentences


# Flattening of nested list
def flatten(list_of_lists):
    return list(itertools.chain.from_iterable(list_of_lists))

# Getting sentences from paragraphs

def tokenize_sent(task):
  filter_quotes_and_questions=[]
  for i in task:
    filter_quotes_and_questions.append(Preprocessing(i))
  return  filter_quotes_and_questions

# method 2: replacing adjective using wordnet

def replace_by_adjectives(text):
  def get_adjectives(text):
      blob = TextBlob(text)
      return [word for (word,tag) in blob.tags if tag == "JJ"]

  adjectives=[]
  F=[]
  # T=[]
  antonyms = []

  for sentence in text:
    r=get_adjectives(sentence)
    adjectives.append(r)

    for i in adjectives:
      if len(i)==0:
        F.append({"question":sentence,"context":sentence,"answer":"True"})
        antonyms.clear()
        adjectives.clear()
      else:
        if len(i)==1:
          for j in i:
            for syn in wordnet.synsets(j):
                for lm in syn.lemmas():
                    if lm.antonyms():
                        antonyms.append(lm.antonyms()[0].name()) #adding into antonyms
            if len(antonyms) >0:
              import random
              random=random.choice(antonyms)
              new_text=sentence.replace(j,random)
              F.append({"question":new_text,"context":sentence,"answer":"False"})
            else:
              F.append({"question":sentence,"context":sentence,"answer":"True"})
        else:
          import random
          randoms=random.choice(i)
          for syn in wordnet.synsets(randoms):
                for lm in syn.lemmas():
                    if lm.antonyms():
                        antonyms.append(lm.antonyms()[0].name()) #adding into antonyms
          if len(antonyms) >0:
            random=random.choice(antonyms)
            new_text=sentence.replace(randoms,random)
            F.append({"question":new_text,"context":sentence,"answer":"False"})
          else:
            F.append({"question":sentence,"context":sentence,"answer":"True"})
        antonyms.clear()
        adjectives.clear()
  return F

# Method 3: replacing NER using Sense2vec, it will replace only one entity at a time

def generate_false_ner_single(text):
  docs = (spacy_nlp.pipe(text, disable=["parser"]))
  text1= spacy_nlp(text)
  Entities=[]
  for word in text1.ents:
      Entities.append(word)
  NER=[] 
  for i in Entities:                        
      NER.append(str(i))
  random.shuffle(NER)
  distractors = []
  True_sen=[]
  False_sen=[]
  for index,i in enumerate(NER):
        originalword=i
        word = originalword.lower()
        word = word.replace(" ", "_")
        sense = s2v.get_best_sense(word)
        if sense== None:
          index+=1
        else:
          most_similar = s2v.most_similar(sense, n=5)
          for each_word in most_similar:
            append_word = each_word[0].split("|")[0].replace("_", " ")
            if append_word not in distractors and append_word != originalword:
                distractors.append(append_word)
          break 
  if len(distractors)==0:
    True_sen.append(text)
  else:
    replace_word=random.choice(distractors)
    False_sentence=text.replace(i,replace_word)
    False_sen.append(False_sentence)
  return True_sen,False_sen

def replace_by_ner_single(checking):
  T=[]
  for i in checking:
    a=generate_false_ner_single(i)
    yes = ' '.join([str(elem) for elem in a[0]])
    no=' '.join([str(elem) for elem in a[1]])
    if len(a[0])==0:
      T.append({'question':no,'context':i,'answer':'False'})
    else:
      T.append({'question':yes,'context':i,'answer':'True'})
  return T

# not generating false sentences

def not_replacing(text):
  same_sent=[]
  for i in text:
    same_sent.append({'question':i,'context':i,'answer':'True'})
  return same_sent

def create_list(chintan):                                   # it will create parts of lists from the original list and then will apply generated lists to all methods
  # fourth=chintan[0:int(0.25*len(chintan))]
  second=chintan[int(0.8*len(chintan)):]
  # third=chintan[int(0.7*len(chintan)):]
  first=chintan[0:int(0.8*len(chintan))]
  return first,second

  # gpt2_array=replace_using_gpt2(first)
  # adjective_array=replace_by_adjectives(second)
  # ner_array=replace_by_ner(third)
  # not_replacing_array=not_replacing(fourth)

def final(ner_array,adjective_array):
  final_list=ner_array+adjective_array
  return final_list

def generate_false_ner_all(text):            # it will replace all NER available in text at a time
  True_sen=[]
  False_sen=[]
  Sense=[]
  text1= spacy_nlp(text)
  Entities=[]
  for word in text1.ents:
      Entities.append(word)
  NER=[] 
  for i in Entities:                        
      NER.append(str(i))
  if len(NER)==0:
    True_sen.append(text)
  else:
    for index,i in enumerate(NER):
      originalword=i
      word = originalword.lower()
      word = word.replace(" ", "_")
      sense = s2v.get_best_sense(word)    
      if sense== None:
        index+=1
      else:
        Sense.append(sense)
        distractors = []
        most_similar = s2v.most_similar(sense, n=10)
        for each_word in most_similar:
          append_word = each_word[0].split("|")[0].replace("_", " ")
          if append_word not in distractors and append_word != originalword:
              distractors.append(append_word)
        replace_word=random.choice(distractors)
        text=text.replace(i,replace_word)
    if len(Sense)==0:
      True_sen.append(text)
    else:
      False_sen.append(text)
  return True_sen,False_sen

def replace_by_ner_all(checking):
  T=[]
  for i in checking:
    a=generate_false_ner_all(i)
    yes = ' '.join([str(elem) for elem in a[0]])
    no=' '.join([str(elem) for elem in a[1]])
    if len(a[1])!=0:
      T.append({'question':no,'context':i,'answer':'False'})
    else:
      T.append({'question':yes,'context':i,'answer':'True'})
  return T

def generate_True_False_statement(para):            #  will generate which of following statement is True?- it will have 3 False statements and 1 true statement
  Final_list=[]
  for text in para:
    True_False_sen=[]                
    Bool_questions=[]
    Bool=[]
    True_sen=[]
    False_sen=[]
    text1= spacy_nlp(text)
    Entities=[]
    for word in text1.ents:
        Entities.append(word)
    NER=[] 
    for i in Entities:                        
        NER.append(str(i))
    if len(NER)==0:
      True_sen.append(text)
    else:
      # random.shuffle(NER)  
      Sense=[]
      for index,i in enumerate(NER):
            originalword=i
            word = originalword.lower()
            word = word.replace(" ", "_")
            sense = s2v.get_best_sense(word)
            if sense== None:
              index+=1
            else:
              distractors = []
              Sense.append(sense)
              most_similar = s2v.most_similar(sense, n=3)
              for each_word in most_similar:
                append_word = each_word[0].split("|")[0].replace("_", " ")
                if append_word not in distractors and append_word != originalword:
                    distractors.append(append_word)
                    for distractor in distractors:
                      replace_word=distractor
                      False_sentence=text.replace(i,replace_word)
                      False_sen.append(False_sentence)
                index+=1
      if len(Sense)==0:
        True_sen.append(text)
      else:
        Bool=[]
        for j in False_sen:
          if j not in Bool and j!=text:
            Bool.append(j)
        if len(Bool)>2:
          Bool_questions.extend(random.sample(Bool,3))
          Bool_questions.append(text)
        elif len(Bool)==2:
          Bool_questions.extend(random.sample(Bool,2))
          Bool_questions.append(text)
        else:
          Bool_questions.extend(random.sample(Bool,1))
          Bool_questions.append(text)
        # else:
        #   return None
        random.shuffle(Bool_questions)
        True_False_sen.append([{'question':'Which of the following statement is True?'},{'statements':Bool_questions},{'original_statement':text}])
        output=flatten(True_False_sen)
        Final_list.append(output)
  return Final_list

def delete_none(false_statement):
  after_delete_none= [i for i in false_statement if i is not None]
  # for i in false_statement:
  #   if len(i)==0:
  #     del i
  #   else:
  #     after_delete_none.append(i)
  return after_delete_none

def Pipeline(text):
  a=sent_tokenize(text)               # get sentences from text as a paragraph
  b=tokenize_sent(a)                  # preprocessing sentences and apply summarizer
  c=flatten(b)                        # Flattening of nested list
  # d=create_list(c)                    # get no. of list for apply different methods 
  # d=replace_using_gpt2(c[0])
  # e=replace_by_ner_all(d[0])          # use NER method to replace all NER at once
  # f=replace_by_adjectives(d[1])       # use adjective antonym method to get opposite words
  d= generate_True_False_statement(c)   #  will generate which of following statement is True?- it will have 3 False statements and 1 true statement
  # e=delete_none(d)                      # delete None means []                     
  return d


In [5]:
import json
import requests
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/6375e9fce0b9ee920501a4f2")
resp= request.json()
file_path=(resp["data"]["file_path"])
_id=(resp["data"]["_id"])
number_of_question=(resp["data"]["number_of_question"])
file_type=(resp["data"]["file_type"])
if file_type == "pdf":
  FileName=file_path
  doc = aw.Document(FileName)
  doc.save("File.docx")
  Parse=parser.from_file("File.docx")
  data=[]
  if not Parse['content'] is None:
    for i in (Parse['content'].strip().split('\n')): 
      if len(i.split())<5:
        pass
      else: 
        data.append(i)
    Text=data[1:-1]
  else:
    Text = None
else:
  FileName=file_path
  Parse=parser.from_file(FileName)
  data=[] 
  if not Parse['content'] is None:
    for i in (Parse['content'].strip().split('\n')): 
      if len(i.split())<5:
        pass
      else: 
        data.append(i)
    Text=data
    
    get_paragraph = []
    for index,k in enumerate(Text):
      get_paragraph.append({"paragraph_no":index+1,"paragraph":k})
  else:
    Text = None
if Text == None or len(Text) == 0:
  print("please enter valid file")
else:
  print("valid file")

2022-11-17 11:55:43,134 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2022-11-17 11:55:43,678 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-11-17 11:55:44,071 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


valid file


In [6]:
Text

["The economy of India is a middle income developing market economy.[39] It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP). According to the International Monetary Fund (IMF), on a per capita income basis, India ranked 142nd by GDP (nominal) and 128th by GDP (PPP).[40] From independence in 1947 until 1991, successive governments promoted protectionist economic policies, with extensive state intervention and economic regulation. This is characterised as dirigism, in the form of the License Raj.[41][42] The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India.[43][44] Since the start of the 21st century, annual average GDP growth has been 6% to 7%,[39] and from 2013 to 2018, India was the world's fastest growing major economy, surpassing China.[45][46] Historically, India was the largest economy in the world for most of the two millennia from the 1st 

In [7]:
generate_ques=int(number_of_question)

In [8]:
# true_false=[]
# for i in Text:
#   outputs=Pipeline(i)
#   true_false.append(outputs)

pool = Pool(processes=2)
outputs = pool.map(Pipeline, Text)
outputs

# x=1
# Final_list=[]  
# for i in outputs:
#   Final_new=[]
#   for j in i:
#     j['Paragraph_no']=x
#     Final_new.append(j)
#   x+=1
#   Final_list.append(Final_new)

Boolean_Questions= flatten(flatten(outputs))

In [9]:
Boolean_Questions

[{'question': 'Which of the following statement is True?'},
 {'statements': ['The economy of India is a middle income developing market economy.',
   'The economy of Malaysia is a middle income developing market economy.',
   'The economy of Indonesia is a middle income developing market economy.',
   'The economy of Bangladesh is a middle income developing market economy.']},
 {'original_statement': 'The economy of India is a middle income developing market economy.'},
 {'question': 'Which of the following statement is True?'},
 {'statements': ['It is the worlds fifth-largest economy by nominal GDP and the third-largest by purchasing power parity PPP.',
   'It is the worlds fifth-largest economy by nominal GDP and the second-largest by purchasing power parity PPP.',
   'It is the worlds fifth-largest economy by nominal GDP and the 3rd-largest by purchasing power parity PPP.',
   'It is the worlds sixth-largest economy by nominal GDP and the third-largest by purchasing power parity PPP

In [10]:
end = time.time()

In [11]:
from datetime import datetime

dt1 = datetime.fromtimestamp(start)
dt2 = datetime.fromtimestamp(end)
total_time = dt2 - dt1
total_time

datetime.timedelta(seconds=128, microseconds=703422)

In [12]:
if generate_ques > len(Boolean_Questions) or generate_ques == 0: 
  print("please give value between 1 to {}".format(len(Boolean_Questions)))
else:
    middle_index = generate_ques
    user_required_questions = Boolean_Questions[:middle_index]
    other_questions = Boolean_Questions[middle_index:]

    
    for question_index, question in enumerate(user_required_questions):
      question["question_id"]= question_index +1

    for question_index, question in enumerate(other_questions):
      question["question_id"]= question_index +1

In [13]:
url="https://generate-questions.devbyopeneyes.com/api/GenerateQuestions" 
headers = {'Content-Type':'application/json','Accept':'application/json'}
post_array ={"id" : _id,
       "questions" : user_required_questions,
       "other_questions" : other_questions,
       "upload_process_time": str(total_time)
  }
status = requests.post(url,headers=headers,data=json.dumps(post_array))

In [14]:
post_array

{'id': '6375e9fce0b9ee920501a4f2',
 'questions': [{'question': 'Which of the following statement is True?',
   'question_id': 1},
  {'statements': ['The economy of India is a middle income developing market economy.',
    'The economy of Malaysia is a middle income developing market economy.',
    'The economy of Indonesia is a middle income developing market economy.',
    'The economy of Bangladesh is a middle income developing market economy.'],
   'question_id': 2},
  {'original_statement': 'The economy of India is a middle income developing market economy.',
   'question_id': 3},
  {'question': 'Which of the following statement is True?', 'question_id': 4},
  {'statements': ['It is the worlds fifth-largest economy by nominal GDP and the third-largest by purchasing power parity PPP.',
    'It is the worlds fifth-largest economy by nominal GDP and the second-largest by purchasing power parity PPP.',
    'It is the worlds fifth-largest economy by nominal GDP and the 3rd-largest by pu

# function for creating multiple false statements from the original

In [15]:
text=['In 2020, pandemic had affected trade and India was the worlds 14th-largest importer and the 21st-largest exporter.']

In [18]:
demo=generate_True_False_statement(text)
demo

[[{'question': 'Which of the following statement is True?'},
  {'statements': ['In 2020, pandemic had affected trade and India was the worlds 14th-largest importer and the 21st-largest exporter.',
    'In 2025, pandemic had affected trade and India was the worlds 14th-largest importer and the 21st-largest exporter.',
    'In 2020, pandemic had affected trade and Bangladesh was the worlds 14th-largest importer and the 21st-largest exporter.',
    'In 2020, pandemic had affected trade and India was the worlds 15th-largest importer and the 21st-largest exporter.']},
  {'original_statement': 'In 2020, pandemic had affected trade and India was the worlds 14th-largest importer and the 21st-largest exporter.'}]]